# Check your environment
In this Notebook, you check the correct setup of your environment. You load the environment using the file _.env_. You have to create the file yourself. We did provided a template to use for the file. The template is called [template-env](file://./template-env).

In the next code block, you import all available variables in the _.env_ file into the environment files. 

In [ ]:
import os

from dotenv import load_dotenv

if load_dotenv(dotenv_path="./.env"):
    print("All environment variables are loaded")
else:
    print("We cannot find the file '.env' in the root of your project. Please use the template 'template-env' to create one. "
          "Run this cell again to retry the loading of the file. ")

Python notebooks has a simple command to show all the system variables. Go ahead, try it out. Add a new code cell and run the following command in the code cell.

```Jupyter
# Print the environment variables
%env

```

The next code block provides some utility function to check for existing keys that we require. We check for three variables:
```text
OPENAI_API_KEY
HUGGINGFACEHUB_API_TOKEN
WEAVIATE_API_KEY
WEAVIATE_CLUSTER_URL
```

Allthough you can run everything on your local machine, we recommend getting used to working with remote services. Using the remote services makes you less dependend on having all the different services running on your local machine. Below are are a few links that help you to obtain the right keys.

- [OpenAI API Key](https://beta.openai.com/account/api-keys)
- [HuggingFace API Key](https://huggingface.co/settings/tokens)
- [Weaviate console to create a cluster](https://console.weaviate.cloud/)

In [ ]:
def verify_environment_keys() -> dict:
    """Parse known environment variables and return their availability in a dict"""
    return {
        "OPENAI_API_KEY": verify_environment_key("OPENAI_API_KEY", "OpenAI"),
        "HUGGINGFACEHUB_API_TOKEN": verify_environment_key("HUGGINGFACEHUB_API_TOKEN", "Hugging Face"),
        "WEAVIATE_API_KEY": verify_environment_key("WEAVIATE_API_KEY", "Weaviate Key"),
        "WEAVIATE_CLUSTER_URL": verify_environment_key("WEAVIATE_CLUSTER_URL", "Weaviate Cluster")
    }

def verify_environment_key(key: str, remote_system: str) -> bool:
    """Verify if the provided key exists in the environment. Returns a bool indicating if the key exists"""
    try:
        found_key = os.environ[key]
    except KeyError as e:
        print(f"Please set the key {key} in the .env file if you want to work with {remote_system}")
        found_key = None

    return found_key is not None


In [ ]:
results = verify_environment_keys()
print("\nOverview of all found and missing environment variables:")
for key in results.keys():
    print(f"{'found' if results[key] else 'ERROR':6s} - {key}")


## Check the connection to OpenAI
After providing the environment variable, you can connect to the OpenAI API. The following code block is an example request to OpenAI. Beware, execting these commands will cost money. Especially if you do it a lot of times. We advise to add boundaries to OpenAI for how much money you want to spend as a maximum.

In [ ]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

chat_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What kind of conference is Devoxx?"},
    ]
)

print(chat_response["choices"][0]["message"]["content"])

## Check the connection to HuggingFace
After providing the environment variable, you can connecto the HuggingFace API endpoint. The following code block is an exmaple request to HuggingFace. At HuggingFace you can choose to use open source models. They are available without payment, the amount of requests are limited though.

In [ ]:
from huggingface_hub.inference_api import InferenceApi

hf_repo_id = "sentence-transformers/all-mpnet-base-v2"

hf_client = InferenceApi(
    repo_id=hf_repo_id,
    token=os.getenv("HUGGINGFACEHUB_API_TOKEN")
)

payload = {
            "source_sentence": "Devoxx is a conference for software developers and programmers",
            "sentences": [
                "Truck drivers should go to Devoxx",
                "Java developers have to go to Devoxx",
                "Truckload 2023 is where every Java Programmer should be",
                "Real truck drivers attend Truckload 2023"
            ]
        }

embed_response = hf_client(inputs=payload)

for sentence, score in zip(payload["sentences"],embed_response):
    print(f"similarity score: {score} for sentence '{sentence}'")

If you want to run the HuggingFace from your own machine, please run the following code block as well.

In [ ]:
import sentence_transformers
import numpy as np
from numpy.linalg import norm

hf_model = sentence_transformers.SentenceTransformer(model_name_or_path="sentence-transformers/all-mpnet-base-v2")

embed_source = hf_model.encode(payload["source_sentence"], convert_to_numpy=True)
embed_sentences = hf_model.encode(payload["sentences"], convert_to_numpy=True)

cosine = np.dot(embed_sentences,embed_source)/(norm(embed_sentences, axis=1)*norm(embed_source))

for sentence, score in zip(payload["sentences"],cosine):
    print(f"similarity score: {score} for sentence '{sentence}'")


## Check the connection to Weaviate
You need to store vectors later on, one way of storing them is using the Weaviate vector database. If you want to use Weaviate, you need to create a cluster and copy the custer url as well as the api key to the _.env_ file. Next you check if the connection works.

In [ ]:
import weaviate

auth_config = weaviate.AuthApiKey(api_key=os.getenv('WEAVIATE_API_KEY'))  # Replace w/ your Weaviate instance API key

# Instantiate the client with the auth config
weaviate_client = weaviate.Client(
    url=os.getenv('WEAVIATE_CLUSTER_URL'),  # Replace w/ your endpoint
    auth_client_secret=auth_config
)

if weaviate_client.is_ready():
    print("You are connected to Weaviate")
else:
    print("Something is wrong with your connection to Weaviate")